<a href="https://colab.research.google.com/github/parthnijh/intro-to-pytorch/blob/main/Pytorch_training_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df=pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [2]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

# train test split


In [3]:
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)


In [4]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [5]:

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [15]:
import torch
X_train_tensor = torch.from_numpy(X_train).float()
X_test_tensor = torch.from_numpy(X_test).float()
y_train_tensor = torch.from_numpy(y_train).float()
y_test_tensor = torch.from_numpy(y_test).float()

# Defining the model

In [7]:
class MySimpleNN():
  def __init__(self,X):
    self.weights=torch.rand(X.shape[1],1,dtype=torch.float64,requires_grad=True)
    self.bias=torch.zeros(1,dtype=torch.float64,requires_grad=True)

  def forward(self,X):
    z=torch.matmul(X,self.weights)+self.bias
    y_pred=torch.sigmoid(z)
    return y_pred

  def loss_function(self,ypred,y):
    epsilon=1e-7
    ypred=torch.clamp(ypred,epsilon,1-epsilon)
    loss=-(y_train_tensor*torch.log(ypred)+(1-y_train_tensor)*torch.log(1-ypred)).mean()
    return loss




In [8]:
# Important parameters

learning_rate=0.1
epochs=25

# training a pipeline

In [9]:
#create a model
model=MySimpleNN(X_train_tensor)

#forward pass
#loss calc
#back pass
#update weights and bias
for epoch in range(epochs):
  ypred=model.forward(X_train_tensor)
  loss=model.loss_function(ypred,y_train_tensor)
  loss.backward()
  with torch.no_grad():
    model.weights-=learning_rate*model.weights.grad
    model.bias-=learning_rate*model.bias.grad
  print(model.weights)
  model.weights.grad.zero_()
  model.bias.grad.zero_()
  print(f"Epoch {epoch+1} Loss :{loss.item()}")


tensor([[0.2853],
        [0.8084],
        [0.6020],
        [0.8908],
        [0.1466],
        [0.8368],
        [0.7600],
        [0.4740],
        [0.0292],
        [0.7469],
        [0.5107],
        [0.8070],
        [0.3602],
        [0.5638],
        [0.9105],
        [0.7147],
        [0.1456],
        [0.3449],
        [0.6654],
        [0.3961],
        [0.9158],
        [0.4442],
        [0.4170],
        [0.1110],
        [0.0745],
        [0.3526],
        [0.7417],
        [0.4140],
        [0.9832],
        [0.2501]], dtype=torch.float64, requires_grad=True)
Epoch 1 Loss :3.6875583129321123
tensor([[0.2634],
        [0.7937],
        [0.5793],
        [0.8699],
        [0.1298],
        [0.8105],
        [0.7328],
        [0.4455],
        [0.0128],
        [0.7379],
        [0.4922],
        [0.8044],
        [0.3421],
        [0.5478],
        [0.9055],
        [0.6947],
        [0.1290],
        [0.3219],
        [0.6617],
        [0.3820],
        [0.8912],
       

# Evaluation

In [10]:
with torch.no_grad():
  ypred=model.forward(X_test_tensor)
  ypred = (ypred > 0.9).float()
  accuracy = (ypred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5867959260940552


In [11]:
import torch.nn as nn
class MySimpleNN(nn.Module):

  def __init__(self,num_features):
    super().__init__()
    self.linear=nn.Linear(num_features,1)
    self.sigmoid=nn.Sigmoid()

  def forward(self,features):
    out=self.linear(features)
    out=self.sigmoid(out)

    return out

  def loss_function(self,ypred,y):
    epsilon=1e-7
    ypred=torch.clamp(ypred,epsilon,1-epsilon)
    loss=-(y_train_tensor*torch.log(ypred)+(1-y_train_tensor)*torch.log(1-ypred)).mean()
    return loss




In [16]:
#create a model
model=MySimpleNN(X_train_tensor.shape[1])

#forward pass
#loss calc
#back pass
#update weights and bias
for epoch in range(epochs):
  ypred=model(X_train_tensor)
  loss=model.loss_function(ypred,y_train_tensor)
  loss.backward()
  with torch.no_grad():
    model.linear.weight-=learning_rate*model.linear.weight.grad
    model.linear.bias-=learning_rate*model.linear.bias.grad
  print(model.linear.weight)
  model.linear.weight.grad.zero_()
  model.linear.bias.grad.zero_()
  print(f"Epoch {epoch+1} Loss :{loss.item()}")

Parameter containing:
tensor([[-0.1148, -0.1057, -0.1087,  0.1598, -0.0078, -0.1732, -0.0976,  0.0708,
         -0.1621,  0.1799,  0.0078, -0.0080, -0.0224, -0.0160, -0.1257,  0.1570,
         -0.1389,  0.0069, -0.0705, -0.0192,  0.1335, -0.1467, -0.1319,  0.1109,
         -0.1618,  0.0122,  0.0008,  0.1259,  0.1041,  0.0604]],
       requires_grad=True)
Epoch 1 Loss :0.7263135313987732
Parameter containing:
tensor([[-0.1125, -0.0997, -0.1062,  0.1623, -0.0042, -0.1695, -0.0932,  0.0748,
         -0.1578,  0.1810,  0.0118, -0.0030, -0.0185, -0.0128, -0.1211,  0.1598,
         -0.1356,  0.0106, -0.0675, -0.0176,  0.1359, -0.1412, -0.1294,  0.1132,
         -0.1584,  0.0143,  0.0038,  0.1288,  0.1058,  0.0610]],
       requires_grad=True)
Epoch 2 Loss :0.7187357544898987
Parameter containing:
tensor([[-0.1111, -0.0943, -0.1047,  0.1638, -0.0013, -0.1670, -0.0901,  0.0775,
         -0.1544,  0.1817,  0.0148,  0.0016, -0.0156, -0.0105, -0.1169,  0.1616,
         -0.1331,  0.0134, -0.0648, 